# <div align = 'center' style='color:rgb(250, 50, 250);'> Applied Loan </div>

# Define Libraries

In [1]:
# %% [markdown]
# # Jupyter Notebook Loading Header
#
# This is a custom loading header for Jupyter Notebooks in Visual Studio Code.
# It includes common imports and settings to get you started quickly.
# %% [markdown]
## Import Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from google.cloud import bigquery
from google.cloud import storage
import os
path = r'C:\Users\Dwaipayan\AppData\Roaming\gcloud\legacy_credentials\dchakroborti@tonikbank.com\adc.json'
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = path
client = bigquery.Client(project='prj-prod-dataplatform')

os.environ["GOOGLE_CLOUD_PROJECT"] = "prj-prod-dataplatform"


from datetime import datetime

# %% [markdown]
## Configure Settings
# Set options or configurations as needed

# Example: pd.set_option('display.max_columns', None)


import os
from nbclient import NotebookClient
from nbformat import read
import time


### Query Logic Documentation

This query constructs a comprehensive customer loan profile by integrating **onboarding details**, **loan application data**, **risk assessments**, and **behavioral metrics**. Below is a breakdown of its logic:

---

#### **1. Core Components**
- **Source Tables**: 
  - Loan data (`risk_credit_mis.loan_master_table`, `dl_loans_db_raw.tdbk_digital_loan_application`).
  - Customer data (`dl_customers_db_raw.tdbk_customer_mtb`, `dl_customers_db_raw.tdbk_cust_profile_mtb`).
  - Reference/lookup tables (e.g., `tdbk_loan_lov_mtb`, `tdbk_industry_list_mtb`).
  - External data (Credolab scores, Trusting Social scores, CIC bureau data).

- **Output**:  
  A distinct record per `digitalLoanAccountId` with 100+ columns spanning:
  - Customer demographics
  - Loan characteristics
  - Risk flags/scores
  - Delinquency metrics
  - Employment/income details

---

#### **2. Key CTEs (Common Table Expressions)**
CTEs preprocess data for efficiency:

| CTE Name                  | Purpose                                                                 |
|---------------------------|-------------------------------------------------------------------------|
| `educationtype`           | Maps education level to loans via `tdbk_loan_purpose`.                 |
| `ref_type`                | Gets relationship types (primary/secondary) for loan references.       |
| `user_type_min`           | Classifies customers (e.g., `New Applicant`, `Repeat Applicant`).     |
| `deliquency`              | Calculates delinquency flags (FPD10, FPD30, etc.).                    |
| `cust_info`               | Flags loan stages (e.g., `flg_applied_loan`, `flg_disbursed_loan`).   |
| `default_outstanding_principal` | Tracks outstanding principal during delinquency events.          |
| `credo_score_static_insght`| Aggregates Credolab scores from multiple sources.                     |

---

#### **3. Column Logic Highlights**
- **Customer Demographics (Onboarding)**:
  ```sql
  DATE_DIFF(onboarding_date, dob, YEAR) AS onb_age,  -- Age at onboarding
  COALESCE(email, loan_email) AS ln_email           -- Prioritizes loan email
  ```
- **Loan Details**:
  ```sql
  CASE WHEN loan_type = 'Flex-up' THEN startApplyDateTime 
       ELSE termsAndConditionsSubmitDateTime END AS loan_submission_date
  ```
- **Risk Flags**:
  ```sql
  CASE WHEN cddRejectReason IS NOT NULL THEN 1 ELSE 0 END AS flg_cdd_reject_flag
  ```
- **Employment/Income**:
  ```sql
  CASE WHEN employment_type = 'Employed' AND MOD(customerId, 10) < 3 
       THEN 'Employed - Govt. Employee' 
       ELSE employment_type END AS ln_employment_type_new
  ```
- **Delinquency Metrics**:
  ```sql
  CASE WHEN min_inst_def30 >= 1 THEN 1 ELSE 0 END AS flg_mature_fpd30
  ```

---

#### **4. Complex Transformations**
- **Nature of Work Mapping**:  
  Maps 50+ raw values to standardized categories (e.g., `'Architect' → 'Licensed Professional'`).
  ```sql
  CASE purpose_nature_of_work_name 
    WHEN 'Architect' THEN 'Licensed Professional'
    ... 
  END AS ln_nature_of_work_new
  ```
- **Industry Cleaning**:  
  Filters irrelevant industries and standardizes names (e.g., `'Fin Tech' → 'Financial Services'`).
  ```sql
  CASE WHEN industry IN ('Gambling', 'FX Dealer') THEN NULL 
       ... 
  END AS ln_industry_new
  ```

- **Dynamic User-Type Classification**:  
  Uses disbursement history to classify applicants:
  ```sql
  CASE 
    WHEN first_disbursement_date > application_date THEN 'New Applicant'
    ... 
  END AS ln_loan_level_user_type
  ```

---

#### **5. Joins & Filters**
- **Critical Joins**:
  - `loan_master_table` → `tdbk_digital_loan_application` (loan details).
  - `tdbk_customer_mtb` → `tdbk_cust_profile_mtb` (demographics).
  - Delinquency CTEs via `loanAccountNumber`.
- **Filter**:  
  `QUALIFY ROW_NUMBER() OVER (PARTITION BY digitalLoanAccountId) = 1` ensures **one record per loan**.

---

#### **6. Special Notes**
- **Timezone Handling**:  
  `datetime(timestamp_column, 'Asia/Manila')` converts timestamps to Philippine time.
- **Null Handling**:  
  Uses `COALESCE()` extensively (e.g., addresses, emails).
- **Flags**:  
  Binary flags (e.g., `flg_disbursed_loan`) simplify loan state tracking.
- **Credolab Scores**:  
  Prioritizes the earliest score per loan from multiple sources.

---

### Summary
This query builds a 360° view of customers and loans by:
1. **Structuring data** through 15+ CTEs.
2. **Enriching** raw fields with business logic (e.g., risk flags, cleaned categories).
3. **Resolving data quality issues** via `COALESCE` and conditional mappings.
4. **Optimizing performance** with window functions for deduplication.

Output columns are prefixed (`onb_` for onboarding, `ln_` for loan) for clarity.

# Constant

In [2]:

from datetime import datetime, timedelta

# Get current date
current_date = datetime.now().date()

# Subtract 1 day
previous_date = current_date - timedelta(days=1)

print("Current Date:", current_date)
print("Previous Date:", previous_date)



OBDATE = previous_date
print(f"The observation date is: {OBDATE}")
print(f"{OBDATE.strftime('%Y%m%d')}")


# Constants
BUCKET_NAME = "prod-asia-southeast1-tonik-aiml-workspace"
CLOUDPATH = "DC/Feature_Mart/Data"
CURRENT_DATE = datetime.now().strftime("%Y%m%d")


Current Date: 2025-12-24
Previous Date: 2025-12-23
The observation date is: 2025-12-23
20251223


# Query

In [ ]:
query = f"""  
create or replace table `risk_mart.applied_loans` as
select distinct customerId,
digitalLoanAccountId,
loanAccountNumber,
onboarding_datetime onb_tsa_onboarding_datetime,
first_name onb_first_name,
middle_name onb_middle_name,
last_name onb_last_name,
onb_age,
gender onb_gender,
onb_mobile_no,
email onb_email,
email_verified_flag onb_email_verified_flag,
place_of_birth onb_place_of_birth,
doctype onb_doc_type,          
onb_country,
onb_province,
onb_city,
onb_barangay,
onb_postalcode,
onb_latitude,
onb_longitude,
onb_osversion,        
kyc_status onb_kyc_status,
datetime(timestamp(kyc_status_change_dt), 'Asia/Manila') onb_kyc_status_upgrade_datetime,
--kyc_status_change_dt onb_kyc_status_upgrade_datetime,
user_type ln_user_type,
ln_loan_level_user_type ln_loan_level_user_type_old,
--ln_loan_level_user_type_v1 ln_loan_level_user_type,
if(loan_type in ('Flex-up','Reloan') ,'1_Repeat Applicant',ln_loan_level_user_type_v1) ln_loan_level_user_type,
loan_type ln_loan_type,
loan_product_type ln_prod_type,
flg_applied_loan ln_loan_applied_flag,
fatca ln_facta_flag,
flg_reject_rule_check ln_dl_rule_reject_flag,
ln_taran_rule_reject_flag,
flg_taran_reject_flag ln_taran_scorecard_reject_flag,
flg_cdd_reject_flag ln_cdd_reject_flag,
ln_marked_underwriter_check_flag,
flg_hard_reject_flag ln_underwriting_reject_flag, 
if(flg_disbursed_loan=1,1,flg_loan_approved_not_disbursed) ln_final_approved_flag,
flg_disbursed_loan ln_disb_flag,
flg_loan_approved_not_disbursed ln_approved_not_disb_flag,
loan_submission_date ln_appln_submit_datetime,
disbursementDateTime ln_disb_dtime,
loan_chosen_principal ln_chosen_principal,
loan_chosen_tenor ln_chosen_tenor,
loan_approved_principal ln_approved_principal,
loan_approved_tenor ln_approved_tenor,
main.onb_cnt_ongoing_loans ln_cnt_ongoing_tdb_loans,
onb_tot_ongoing_loans_emi ln_tot_ongoing_tdb_loans_emi,
loan_purpose ln_purpose,
vas_opted_flag ln_vas_opted_flag,
vas_used_flag ln_vas_used_flag,
loan_age ln_age,
loan_mobile_no ln_mobile_no,
loan_alternate_mobile_no ln_alt_mobile_no,
osversion_v2 ln_osversion,
brand ln_brand,
ln_os_type,
ln_model,
coalesce(ln_email,email) ln_email,
coalesce(loan_address,addressline) ln_address,
coalesce(loan_province,province) ln_province,
coalesce(loan_city,city) ln_city,
coalesce(loan_barangay,barangay) ln_barangay,
coalesce(cast(loan_postal_code as string),postalcode) ln_postal_code,
loan_latitude ln_latitude,
loan_longitude ln_longitude,
loan_docType ln_doc_type,
loan_docNumber ln_doc_number,
if(loan_marital_status='Live-in Partner','With a Live-in Partner',loan_marital_status) ln_marital_status,
--case when loan_dependents_count in ('No dependents') then null else loan_dependents_count end ln_cnt_dependents,
loan_dependents_count ln_cnt_dependents,
loan_education_level ln_education_level,
coalesce(purpose_source_of_fund_name,main.source_funds) ln_source_funds,
--coalesce(source_funds_new.source_funds_new,main.source_funds) ln_source_funds_new,
case when coalesce(purpose_source_of_fund_name,main.source_funds) in ('Others') then null
when coalesce(purpose_source_of_fund_name,main.source_funds) in ('Commissions') then null else coalesce(purpose_source_of_fund_name,main.source_funds) end as ln_source_funds_new,
coalesce(purpose_employment_status_name,employmentstatus) ln_employment_type,
--coalesce(employment_type_new.employment_type_new,main.employment_type) ln_employment_type_new,
case when coalesce(purpose_employment_status_name,employmentstatus)='Employed' and Mod(cast(customerId as int64),10)<3 then 'Employed - Govt. Employee/Govt. Official' 
when coalesce(purpose_employment_status_name,employmentstatus)='Employed' and Mod(cast(customerId as int64),10)>2 then'Employed - Private Employee' 
else coalesce(purpose_employment_status_name,employmentstatus) end as ln_employment_type_new,
-- case when upper(coalesce(purpose_employment_status_name,main.employment_type)) like'%EMPLOYED%' and Mod(cast(customerId as int64),10)<3 then 'Employed - Govt. Employee/Govt. Official'
-- when upper(coalesce(purpose_employment_status_name,main.employment_type)) like'%EMPLOYED%' and Mod(cast(customerId as int64),10)>2 then'Employed - Private Employee'
-- else coalesce(purpose_employment_status_name,main.employment_type) end as ln_employment_type_new,
--main.employment_type_new ln_employment_type_new,
coalesce(purpose_nature_of_work_name,natureofwork) ln_nature_of_work,
--coalesce(nature_of_work_new.nature_of_work_new,main.nature_of_work) ln_nature_of_work_new,
case when coalesce(purpose_nature_of_work_name,natureofwork) in ('Admin/Clerical Staff/Officer') then  null
when coalesce(purpose_nature_of_work_name,natureofwork) in ('Architect/Interior Decorator') then  'Licensed Professional - Others'
when coalesce(purpose_nature_of_work_name,natureofwork) in ('Bank Employee') then  null
when coalesce(purpose_nature_of_work_name,natureofwork) in ('Broker/Dealer') then  'Broker/Dealer'
when coalesce(purpose_nature_of_work_name,natureofwork) in ('Call Center Agent/Tele Marketer') then  null
when coalesce(purpose_nature_of_work_name,natureofwork) in ('Cashier/Food Server/Waiter') then  null
when coalesce(purpose_nature_of_work_name,natureofwork) in ('Consultant') then  null
when coalesce(purpose_nature_of_work_name,natureofwork) in ('Doctor/Dentist/Medical Professional') then  'Licensed Professional - Others'
when coalesce(purpose_nature_of_work_name,natureofwork) in ('Engineer') then  'Licensed Professional - Others'
when coalesce(purpose_nature_of_work_name,natureofwork) in ('Entertainer/Artist') then  null
when coalesce(purpose_nature_of_work_name,natureofwork) in ('Foreign Ambassador/Diplomat') then  null
when coalesce(purpose_nature_of_work_name,natureofwork) in ('Gov Employee') then  'Govt Employee'
when coalesce(purpose_nature_of_work_name,natureofwork) in ('Govt Employee') then  'Govt Employee'
when coalesce(purpose_nature_of_work_name,natureofwork) in ('Insurance Agent/Financial Advisor') then  null
when coalesce(purpose_nature_of_work_name,natureofwork) in ('IT Professional') then  null
when coalesce(purpose_nature_of_work_name,natureofwork) in ('Lawyer') then  'Licensed Professional - Lawyer'
when coalesce(purpose_nature_of_work_name,natureofwork) in ('Military/Armed Forces/Police/Navy/Airforce') then  'Government - Uniformed Service'
when coalesce(purpose_nature_of_work_name,natureofwork) in ('Newscaster/Media Personality') then  null
when coalesce(purpose_nature_of_work_name,natureofwork) in ('Pilot/Airline Crew') then  null
when coalesce(purpose_nature_of_work_name,natureofwork) in ('Seafarer') then  null
when coalesce(purpose_nature_of_work_name,natureofwork) in ('Teacher/Instructor/Coach') then  null
when coalesce(purpose_nature_of_work_name,natureofwork) in ('Utility Personnel/Household Help') then  null
when coalesce(purpose_nature_of_work_name,natureofwork) in ('Accountant/Auditor') then  'Licensed Professional - Accountant'
when coalesce(purpose_nature_of_work_name,natureofwork) in ('Utilities (Electricity Gas and Water)') then  null
when coalesce(purpose_nature_of_work_name,natureofwork) in ('Wholesale and Retail Trade') then  null
when coalesce(purpose_nature_of_work_name,natureofwork) in ('Sales/Marketing Personnel') then  'Sales/Marketing Personnel'
when coalesce(purpose_nature_of_work_name,natureofwork) in ('Other Professional services') then  'Licensed Professional - Others'
when coalesce(purpose_nature_of_work_name,natureofwork) in ('Other Non Professional Services') then  null
when coalesce(purpose_nature_of_work_name,natureofwork) in ('Owner') then  'Owner'
when coalesce(purpose_nature_of_work_name,natureofwork) in ('Religious Leader') then  'Religious Leader'
when coalesce(purpose_nature_of_work_name,natureofwork) in ('Fire Fighter') then  'Government - Uniformed Service'
when coalesce(purpose_nature_of_work_name,natureofwork) in ('Farmer/Fisherman') then  null
when coalesce(purpose_nature_of_work_name,natureofwork) in ('Debt Collection/Recovery Agents/ Collection Agency Employee') then  'Debt Collection/Recovery Agents/ Collection Agency Employee'
when coalesce(purpose_nature_of_work_name,natureofwork) in ('Court Personnel/Bailiff') then  'Court Personnel/Bailiff'
when coalesce(purpose_nature_of_work_name,natureofwork) in ('Store/Service Manager') then  'Manager/Executive'
when coalesce(purpose_nature_of_work_name,natureofwork) in ('Sales Marketing Professional') then  'Sales/Marketing Personnel'
when coalesce(purpose_nature_of_work_name,natureofwork) in ('Barber / Beauty Salon') then  'Sales/Marketing Personnel'
when coalesce(purpose_nature_of_work_name,natureofwork) in ('Business Owner (MSME)') then  'Owner'
when coalesce(purpose_nature_of_work_name,natureofwork) in ('Business Owner') then  'Owner'
when coalesce(purpose_nature_of_work_name,natureofwork) in ('Delivery Rider/Driver') then  'Delivery Rider/Driver'
when coalesce(purpose_nature_of_work_name,natureofwork) in ('Delivery Rider / Driver') then  'Delivery Rider / Driver'
when coalesce(purpose_nature_of_work_name,natureofwork) in ('Manpower Agency') then  null
when coalesce(purpose_nature_of_work_name,natureofwork) in ('Online Freelancing / Virtual Assistant / Online teacher') then  'Virtual/Online/Remote Employee'
when coalesce(purpose_nature_of_work_name,natureofwork) in ('Online Seller') then  'Virtual/Online/Remote Employee'
when coalesce(purpose_nature_of_work_name,natureofwork) in ('Real Estate agent') then  null
when coalesce(purpose_nature_of_work_name,natureofwork) in ('Skilled Worker') then  'Skilled Worker'
when coalesce(purpose_nature_of_work_name,natureofwork) in ('Technician (Cellphone / Laptop)') then  null
when coalesce(purpose_nature_of_work_name,natureofwork) in ('Vendor (Sari-Sari store Wet market etc)') then  null
when coalesce(purpose_nature_of_work_name,natureofwork) in ('Staff/Rank and File') then  'Staff/Rank and File'
when coalesce(purpose_nature_of_work_name,natureofwork) in ('Officer/Supervisor') then  'Officer/Supervisor'
when coalesce(purpose_nature_of_work_name,natureofwork) in ('Manager/Executive') then  'Manager/Executive'
when coalesce(purpose_nature_of_work_name,natureofwork) in ('Licensed Professional - Lawyer') then  'Licensed Professional - Lawyer'
when coalesce(purpose_nature_of_work_name,natureofwork) in ('Licensed Professional - Accountant') then  'Licensed Professional - Accountant'
when coalesce(purpose_nature_of_work_name,natureofwork) in ('Licensed Professional - Others') then  'Licensed Professional - Others'
when coalesce(purpose_nature_of_work_name,natureofwork) in ('Religious Minister') then  'Religious Minister'
when coalesce(purpose_nature_of_work_name,natureofwork) in ('Government - Elected Official') then  'Government - Elected Official'
when coalesce(purpose_nature_of_work_name,natureofwork) in ('Government - Uniformed Service') then  'Government - Uniformed Service'
when coalesce(purpose_nature_of_work_name,natureofwork) in ('Director/Trustee/Stockholder') then  'Director/Trustee/Stockholder'
when coalesce(purpose_nature_of_work_name,natureofwork) in ('Virtual/Online/Remote Employee') then  'Virtual/Online/Remote Employee'
when coalesce(purpose_nature_of_work_name,natureofwork) in ('Government - Appointed Official') then  'Government - Appointed Official'
when coalesce(purpose_nature_of_work_name,natureofwork) in ('AIR FORCE') then  'Military/Armed Forces/Police/Navy/Airforce'
when coalesce(purpose_nature_of_work_name,natureofwork) in ('NAVY') then  'Military/Armed Forces/Police/Navy/Airforce'
when coalesce(purpose_nature_of_work_name,natureofwork) in ('Admin/Secretarial') then  'Admin/Clerical Staff/Officer'
when coalesce(purpose_nature_of_work_name,natureofwork) in ('Sales/Marketing Professional') then  'Sales/Marketing Personnel'
when coalesce(purpose_nature_of_work_name,natureofwork) in ('COURT-PERSONNEL/BAILIFFS') then  'Court Personnel/Bailiff'
when coalesce(purpose_nature_of_work_name,natureofwork) in ('DEBT COLLECTION/RECOVERY AGENTS/COLLECTION AGENCY EMPLOYEE') then  'Debt Collection/Recovery Agents/ Collection Agency Employee'
when coalesce(purpose_nature_of_work_name,natureofwork) in ('Accountant') then  'Licensed Professional - Accountant' else coalesce(purpose_nature_of_work_name,natureofwork) end as ln_nature_of_work_new,
if(purpose_created_dt>='2025-05-20',purpose_industryName,industry) ln_industry,
case when main.source_funds in ('Remittance','Pension','Inheritance/Retirement Funds','Interest on Deposits and Investments','Allowance') then null 
when if(purpose_created_dt>='2025-05-20',purpose_industryName,industry) in ('Wholesale and Retail Trade') then 'Wholesale and Retail Trade'
when if(purpose_created_dt>='2025-05-20',purpose_industryName,industry) in ('Utilities (Electricity, Gas and Water)') then 'Utilities (Electricity, Gas and Water)'
when if(purpose_created_dt>='2025-05-20',purpose_industryName,industry) in ('Transportation') then 'Transportation'
when if(purpose_created_dt>='2025-05-20',purpose_industryName,industry) in ('Tourism, Hotel and Resorts') then 'Tourism, Hotel and Resorts'
when if(purpose_created_dt>='2025-05-20',purpose_industryName,industry) in ('TeleCo') then 'Telecommunication'
when if(purpose_created_dt>='2025-05-20',purpose_industryName,industry) in ('Telecommunication') then 'Telecommunication'
when if(purpose_created_dt>='2025-05-20',purpose_industryName,industry) in ('Religious Organization') then 'Religious Organization'
when if(purpose_created_dt>='2025-05-20',purpose_industryName,industry) in ('Entertainment and Media') then 'Recreation (Arts, Entertainment/Media, Sports)'
when if(purpose_created_dt>='2025-05-20',purpose_industryName,industry) in ('Real Estate') then 'Real Estate'
when if(purpose_created_dt>='2025-05-20',purpose_industryName,industry) in ('Pawnshop/Remittance Agent/Virtual Currency') then 'Pawnshop/Remittance Agent/Virtual Currency'
when if(purpose_created_dt>='2025-05-20',purpose_industryName,industry) in ('Others') then 'Others'
when if(purpose_created_dt>='2025-05-20',purpose_industryName,industry) in ('Non Government Organization/Foundation') then 'NGOs/NonProfit Organization/Foundation'
when if(purpose_created_dt>='2025-05-20',purpose_industryName,industry) in ('Military and Weaponry Business') then 'Military and Weaponry Business'
when if(purpose_created_dt>='2025-05-20',purpose_industryName,industry) in ('Maritime/Shipping') then 'Maritime/Shipping'
when if(purpose_created_dt>='2025-05-20',purpose_industryName,industry) in ('Manufacturing - Food') then 'Manufacturing'
when if(purpose_created_dt>='2025-05-20',purpose_industryName,industry) in ('Manufacturing - Non Food') then 'Manufacturing'
when if(purpose_created_dt>='2025-05-20',purpose_industryName,industry) in ('Manpower/Manning Agencies') then 'Manpower/Manning Agencies'
when if(purpose_created_dt>='2025-05-20',purpose_industryName,industry) in ('Legal Service/Law Firm') then 'Legal Service/Law Firm'
when if(purpose_created_dt>='2025-05-20',purpose_industryName,industry) in ('Insurance Activities') then 'Insurance Activities'
when if(purpose_created_dt>='2025-05-20',purpose_industryName,industry) in ('Hospital and Medical Services') then 'Hospital and Medical Services'
when if(purpose_created_dt>='2025-05-20',purpose_industryName,industry) in ('Govt. Institution') then 'Govt. Institution'
when if(purpose_created_dt>='2025-05-20',purpose_industryName,industry) in ('Gold, Jewelry and Precious Metals') then 'Gold/Jewelry/Precious Metals/Art/Antique Dealership'
when if(purpose_created_dt>='2025-05-20',purpose_industryName,industry) in ('Online Gaming/Lottery/Casino') then 'Gambling and Betting (Online Gaming, Lottery, Casino)'
when if(purpose_created_dt>='2025-05-20',purpose_industryName,industry) in ('Freight and Cargo') then 'Freight and Cargo'
when if(purpose_created_dt>='2025-05-20',purpose_industryName,industry) in ('Food Industry/Restaurants/Fastfoods') then 'Food Industry/Restaurants/Fastfoods'
when if(purpose_created_dt>='2025-05-20',purpose_industryName,industry) in ('Fin Tech/Financial Services') then 'Fin Tech/Financial Services'
when if(purpose_created_dt>='2025-05-20',purpose_industryName,industry) in ('FX Dealer/Money Changer') then 'FX Dealer/Money Changer'
when if(purpose_created_dt>='2025-05-20',purpose_industryName,industry) in ('Education/Schools') then 'Education/Private Schools'
when if(purpose_created_dt>='2025-05-20',purpose_industryName,industry) in ("Convenience Store/Internet Cafe's") then "Convenience Store/Internet Cafe's"
when if(purpose_created_dt>='2025-05-20',purpose_industryName,industry) in ('Construction') then 'Construction'
when if(purpose_created_dt>='2025-05-20',purpose_industryName,industry) in ('Computer/IT Related Services') then 'Computer/IT Related Services'
when if(purpose_created_dt>='2025-05-20',purpose_industryName,industry) in ('Business Process Outsourcing/Tele Marketing') then 'Business Process Outsourcing/Tele Marketing'
when if(purpose_created_dt>='2025-05-20',purpose_industryName,industry) in ('Beauty and Health Services') then 'Beauty and Health Services'
when if(purpose_created_dt>='2025-05-20',purpose_industryName,industry) in ('Health and Beauty Services') then 'Beauty and Health Services'
when if(purpose_created_dt>='2025-05-20',purpose_industryName,industry) in ('Banking') then 'Banking'
when if(purpose_created_dt>='2025-05-20',purpose_industryName,industry) in ('Architecture/Engineering') then 'Architecture/Engineering'
when if(purpose_created_dt>='2025-05-20',purpose_industryName,industry) in ('Airline/Aviation') then 'Airline/Aviation'
when if(purpose_created_dt>='2025-05-20',purpose_industryName,industry) in ('Agribusiness/Animal Farming/Fishing') then 'Agribusiness/Animal Farming/Fishing'
when if(purpose_created_dt>='2025-05-20',purpose_industryName,industry) in ('Advertising/Sales/Marketing') then 'Advertising/Sales/Marketing'
when if(purpose_created_dt>='2025-05-20',purpose_industryName,industry) in ('Accounting/Bookkeeping') then 'Accounting/Bookkeeping/Auditing'
when if(purpose_created_dt>='2025-05-20',purpose_industryName,industry) is null then null
when if(purpose_created_dt>='2025-05-20',purpose_industryName,industry) in ('Leasing and Rental Services') then null
when if(purpose_created_dt>='2025-05-20',purpose_industryName,industry) in ('Repair, Sanitation and Housekeeping') then null
when if(purpose_created_dt>='2025-05-20',purpose_industryName,industry) in ('Repair, Sanitation and Housekeeping') then null
when if(purpose_created_dt>='2025-05-20',purpose_industryName,industry) in ('Embassy') then null
when if(purpose_created_dt>='2025-05-20',purpose_industryName,industry) in ('Recycling') then null
when if(purpose_created_dt>='2025-05-20',purpose_industryName,industry) in ('Publishing and Printing') then null
when if(purpose_created_dt>='2025-05-20',purpose_industryName,industry) in ('IT services','ONLINE SELLER','business owner','Human Resources','Head of sales','Human Resource Management',
'Digital Marketing','Food services','Manufacture of Leather and Related Products','Manufacture of Other Non-Metallic Mineral Products',
'Businesswoman','Business Owner','Agency Manpower','Advertising and marketing','Manufacture of Coke and Refined Petroleum Products',
'Sales Coffee Trading','Pharmaceutical','Manufacture of Furniture','Software as a Service','Nurse','Manufacture of Rubber Products','Shared Services',
'Waste and recycling solutions','Event Supplier','Fishing and Aquaculture','Manufacture of Motor Vehicles, Trailers and Semi-Trailers',
'Crop and Animal Production, Hunting and Related Service Activities; Forestry and Logging','Publishing Activities',
'Manufacture of Basic Metals','Manufacture of Textiles','Manufacture of Electrical Equipment',
'Water Collection, Treatment and Supply','Water Transport','Manufacture of Basic Pharmaceutical Products and Pharmaceutical Preparation',
'Manufacture of Chemicals and Chemical Products','Insurance, Reinsurance and Pension Funding except Compulsory Social Security',
'Electricity, Gas, Steam and Air Conditioning Supply','Manufacture of Machinery and Equipment, n.e.c.',
'Travel Agency, Tour Operator, Reservation Service and Related Activities','Medical Transcription  Activities',
'Manufacture of Fabricated Metal Products except Machinery and Equipment','Manufacture of Plastic Products',
'Postal and Courier Activities','Real Estate Activities','Printing and Reproduction of Recorded Media',
'Manufacture of Computer, Electronic and Optical Products','Transport via Buses',
'Religious Organization ','Designated NonFinancial Businesses ','Manufacture of Wearing Apparel',
'Warehousing and Support Activities for Transportation','Architectural and Engineering Activities and Related Technical Consultancy',
'Wholesale  and Retail Trade and Repair of Motor Vehicles and Motorcycles','Designated NonFinancial Businesses',
'Human Health Activities except Public Health Activities','Manufacture of Food Products',
'Military and Weaponry Business','Retail Trade except of Motor Vehicles and Motorcycles',
'Land Transport and Transport via Pipelines except  Transport via Buses','Education except Public Education',
'Financial Service Activities except Insurance, Pension Funding and Central Banking','Wholesale Trade except of Motor Vehicles and Motorcycles',
'Accounting, Bookkeeping and Auditing Activities; Tax Consultancy','Accommodation and Food Service Activities',
'NGOs/NonProfit Organization/Foundation ','Mining and Quarrying ','Computer Programming, Consultancy and Related Activities; Information Service Activities',
'Call Center Activities (Voice)','Back-Office Operations Activities (Non-Voice) Except Medical Transcription Activities',
'Nightclubs/Discos/Karaoke Center','Import Export Companies ','Mining and Quarrying','Import Export Companies',
'Gambling and Betting (Online Gaming, Lottery, Casino) ','Other Personal Services Activities','FX Dealer/Money Changer',
'Religious Organization','Securities/Holding Companies ','Car/Boat/Plane/Used Truck/AutoParts Manufacturing/Dealership or Auction Houses') then null
else if(purpose_created_dt>='2025-05-20',purpose_industryName,industry) end as ln_industry_new, 
--purpose_source_of_fund_name,purpose_nature_of_work_name,purpose_employment_status_name,
--coalesce(industry_description_new.industry_description_new,if(pupose_industryid is not null , coalesce(purpose_industryName,if(purpose_created_dt>='2025-05-20',purpose_industryName,industry)),if(purpose_created_dt>='2025-05-20',purpose_industryName,industry))) ln_industry_new,
loan_companyName ln_company_name,
loan_self_dec_income ln_self_dec_income,
loan_salary_scaled_income ln_salary_scaled_income,
loan_ref_type1 ln_ref1_type,
loan_ref_type2 ln_ref2_type,
flg_cic_called cic_called_flag,
flg_cic_hit cic_hit_flag,
ScoreRange cic_score_range,
demo_score old_demo_score,
cic_score old_cic_score,
credo_score old_gen_credo_score,
trusting_social_score ts_trusting_social_score,
bureau_score bu_bureau_score,
c_credo_score,s_credo_score,fu_credo_score,r_credo_score,gen_credo_score,
flg_mature_fpd10 ln_mature_fpd10_flag,
flg_mature_fpd30 ln_mature_fpd30_flag,
flg_mature_fspd_30 ln_mature_fspd30_flag,
flg_mature_fstpd_30 ln_mature_fstpd30_flag,
deffpd10 ln_fpd10_flag,
deffpd30 ln_fpd30_flag,
deffspd30 ln_fspd30_flag,
deffstpd30 ln_fstpd30_flag,
--loan_defFPD30 ln_defFPD30,
loan_defSPD30 ln_spd30_flag,loan_defTPD30 ln_tpd30_flag,
defpd10_outstanding_principal ln_fpd10_os_principal,defpd30_outstanding_principal ln_fpd30_os_principal,
defspd30_outstanding_principal ln_fspd30_os_principal,deftpd30_outstanding_principal ln_fstpd30_os_principal,
credolabRefNumber,
datetime(credo_inquiry_date,'Asia/Manila') credo_inquiry_date,                              
status cust_status_flag,
onb_self_dec_income,
onb_company_name,
cust_status_close_date

from 
(with educationtype AS (
        SELECT
        loan.digitalLoanAccountId,loanAccountNumber, loan.loanPurposeId,   education_id,
        description AS education_level
        FROM  dl_loans_db_raw.tdbk_digital_loan_application loan
        LEFT JOIN  dl_loans_db_raw.tdbk_loan_purpose purpose
        ON   loan.loanPurposeId = purpose.loanPurposeId
        LEFT JOIN dl_loans_db_raw.tdbk_loan_lov_mtb
        ON education_id = id AND module = 'Education'
         ),

ref_type as (with Ref_type1 as (select  digitalLoanAccountId,relationship_id , description AS loan_ref_type1 from dl_loans_db_raw.tdbk_loan_refernce_details A
        LEFT JOIN dl_loans_db_raw.tdbk_loan_lov_mtb B  ON A.relationship_id = B.id
 where refPrefrenceOrder='1' qualify row_number() over (partition by digitalLoanAccountId order by refCreatedDateAndTime desc) =1),
    Ref_type2 as (select  digitalLoanAccountId,relationship_id , description AS loan_ref_type2 from dl_loans_db_raw.tdbk_loan_refernce_details A
        LEFT JOIN dl_loans_db_raw.tdbk_loan_lov_mtb B  ON A.relationship_id = B.id
 where refPrefrenceOrder='2' qualify row_number() over (partition by digitalLoanAccountId order by refCreatedDateAndTime desc) =1)
select Ref_type1.digitalLoanAccountId,loan_ref_type1,loan_ref_type2 from Ref_type1 join Ref_type2 on Ref_type1.digitalLoanAccountId=Ref_type2.digitalLoanAccountId) ,

user_type_min as (select customerId,min(case 
when digitalLoanAccountId is null then '4_Never_start_loan_appln'
when application_submit_date is null and rn1_application_status <=1 then '4_Never_start_loan_appln'
when application_submit_date is not null and rn1_application_status >=1  and rn_disburement_status is null then '3_Applied_Not_Disbursed'
when application_submit_date is not null and rn1_application_status >=1  and rn_disburement_status =1 then '2_New Applicant'
when application_submit_date is not null and rn1_application_status >=1  and rn_disburement_status >1 then '1_Repeat Applicant'

else '5_others' end ) user_type from
(select A.*,B.rn_disburement_status from
(select digitalLoanAccountId,customerId,applicationStatus, coalesce(termsAndConditionsSubmitDateTime,if (new_loan_type ='Flex-up',startApplyDateTime,termsAndConditionsSubmitDateTime)) application_submit_date,disbursementDateTime,
row_number() over (partition by customerId order by coalesce(termsAndConditionsSubmitDateTime,if (new_loan_type ='Flex-up',startApplyDateTime,termsAndConditionsSubmitDateTime))) rn1_application_status,
from risk_credit_mis.loan_master_table) A

left join 

(select digitalLoanAccountId,customerId,applicationStatus, disbursementDateTime,
case when disbursementDateTime is not null then 
row_number() over (partition by customerId order by disbursementDateTime) else 0 end rn_disburement_status,
from risk_credit_mis.loan_master_table where  disbursementDateTime is not null) B on A.digitalLoanAccountId=B.digitalLoanAccountId) 
--order by A.customerId 
group by 1 ),

user_type_loan_level as (select customerId,digitalLoanAccountId,min(case 
when digitalLoanAccountId is null then '4_Never_start_loan_appln'
when application_submit_date is null and rn1_application_status <=1 then '4_Never_start_loan_appln'
when application_submit_date is not null and rn1_application_status >=1  and rn_disburement_status is null then '3_Applied_Not_Disbursed'
when application_submit_date is not null and rn1_application_status >=1  and rn_disburement_status =1 then '2_New Applicant'
when application_submit_date is not null and rn1_application_status >=1  and rn_disburement_status >1 then '1_Repeat Applicant'

else '5_others' end ) user_type from
(select A.*,B.rn_disburement_status from
(select digitalLoanAccountId,customerId,applicationStatus, coalesce(termsAndConditionsSubmitDateTime,if (new_loan_type ='Flex-up',startApplyDateTime,termsAndConditionsSubmitDateTime)) application_submit_date,disbursementDateTime,
row_number() over (partition by customerId order by coalesce(termsAndConditionsSubmitDateTime,if (new_loan_type ='Flex-up',startApplyDateTime,termsAndConditionsSubmitDateTime))) rn1_application_status,
from risk_credit_mis.loan_master_table
where if (new_loan_type ='Flex-up',startApplyDateTime,termsAndConditionsSubmitDateTime) between '2023-01-01' and '{OBDATE}'
) A

left join 

(select digitalLoanAccountId,customerId,applicationStatus, disbursementDateTime,
case when disbursementDateTime is not null then 
row_number() over (partition by customerId order by disbursementDateTime) else 0 end rn_disburement_status,
from risk_credit_mis.loan_master_table where  disbursementDateTime is not null and  if (new_loan_type ='Flex-up',startApplyDateTime,termsAndConditionsSubmitDateTime) between '2023-01-01' and '{OBDATE}') B on A.digitalLoanAccountId=B.digitalLoanAccountId) 

group by 1 ,2 order by customerId),

user_type_loan_level_v1 as (select customerId,digitalLoanAccountId,disbursementDateTime,application_submit_date,rn_disburement_status,first_disbursment_loan,  min(case 
when digitalLoanAccountId is null then '4_Never_start_loan_appln'
when application_submit_date is null and rn1_application_status <=1 then '4_Never_start_loan_appln'
--when application_submit_date is not null and rn1_application_status >=1  and rn_disburement_status is null then '3_Applied_Not_Disbursed'
when application_submit_date is not null and rn1_application_status >=1  and rn_disburement_status =1 and first_disbursment_loan > application_submit_date then '2_New Applicant'
when application_submit_date is not null and rn1_application_status >=1  and rn_disburement_status >=1  then '1_Repeat Applicant'
when application_submit_date is not null and rn1_application_status >=1  and  first_disbursment_loan < application_submit_date  then '1_Repeat Applicant'
when application_submit_date is not null and rn1_application_status >=1  and rn_disburement_status is null then '2_New Applicant'

else '5_others' end ) user_type from
(select A.*,
B.rn_disburement_status ,C.first_disbursment_loan,
from
(select digitalLoanAccountId,customerId,applicationStatus, coalesce(termsAndConditionsSubmitDateTime,if (new_loan_type ='Flex-up',startApplyDateTime,termsAndConditionsSubmitDateTime)) application_submit_date,disbursementDateTime,
row_number() over (partition by customerId order by coalesce(termsAndConditionsSubmitDateTime,if (new_loan_type ='Flex-up',startApplyDateTime,termsAndConditionsSubmitDateTime))) rn1_application_status,
from risk_credit_mis.loan_master_table
where if (new_loan_type ='Flex-up',startApplyDateTime,termsAndConditionsSubmitDateTime) between '2023-01-01' and '{OBDATE}'
) A

left join 

(select digitalLoanAccountId,customerId,applicationStatus, disbursementDateTime,
case when disbursementDateTime is not null then 
row_number() over (partition by customerId order by disbursementDateTime) else 0 end rn_disburement_status,
from risk_credit_mis.loan_master_table where  disbursementDateTime is not null and  credolabRefNumber is not null and if (new_loan_type ='Flex-up',startApplyDateTime,termsAndConditionsSubmitDateTime) between '2023-01-01' and '{OBDATE}') B on A.digitalLoanAccountId=B.digitalLoanAccountId 

left join 

(select min(disbursementDateTime) first_disbursment_loan,customerId,
from risk_credit_mis.loan_master_table where  disbursementDateTime is not null and  if (new_loan_type ='Flex-up',startApplyDateTime,termsAndConditionsSubmitDateTime) between '2023-01-01' and '{OBDATE}'
group by customerId) C on A.customerId=C.customerId
) 
--where customerId in (1668511,2480643,2027516,1802996,2469891,2418795)

group by 1 ,2,3,4,5,6 order by customerId ,application_submit_date),


deliquency as
(select loanAccountNumber,
case when obs_min_inst_def10 >=1 and min_inst_def10 =1 then 1 else 0 end deffpd10,
case when obs_min_inst_def30 >=1 and min_inst_def30 =1 then 1 else 0 end deffpd30,
case when obs_min_inst_def30 >=2 and min_inst_def30  in (1,2) then 1 else 0 end deffspd30,
case when obs_min_inst_def30 >=3 and min_inst_def30 in (1,2,3) then 1 else 0 end deffstpd30,
case when obs_min_inst_def10 >=1 then 1 else 0 end flg_mature_fpd10,
case when obs_min_inst_def30 >=1 then 1 else 0 end flg_mature_fpd30,
case when obs_min_inst_def30 >=2 then 1 else 0 end flg_mature_fspd_30,
case when obs_min_inst_def30 >=3 then 1 else 0 end flg_mature_fstpd_30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data),

cust_info as (
with cust_base as(
select * from `dl_customers_db_raw.tdbk_customer_mtb`
where cust_id is not null
--and status in ('C','A')
--and date(created_dt,'Asia/Manila') >= '2023-01-01'
),
first_loan_application as (
select custId,digitalLoanAccountId,loanaccountnumber,isTermsAndConditionstimestamp,poi1RejectType,loantype from
--(select * from 
`dl_loans_db_raw.tdbk_digital_loan_application`
--where custId in (select cast(cust_id as integer) from cust_base) qualify row_number() over (partition by custId order by created_dt) = 1)
--where --loanType in ('TUL') and coalesce(flowType,'1') <> '2'
),
loan_calculator as (
select custid,digitalLoanAccountId from
(SELECT fla.custid,fla.digitalLoanAccountId,
sum(case when a.applicationStatus='INITIATE' then 1 else 0 end) loan_calculator
 FROM `dl_loans_db_raw.tdbk_status_trace`  a
 join first_loan_application fla
 on fla.digitalLoanAccountId=a.digitalLoanAccountId
 group by 1,2)
where loan_calculator>0
),
disbursed_undisbursed_loan as (
SELECT fla.custid,fla.digitalLoanAccountId,
 FROM `dl_loans_db_raw.tdbk_status_trace`  a
 join first_loan_application fla
 on fla.digitalLoanAccountId=a.digitalLoanAccountId
 where a.applicationStatus ='APPROVED'
 group by 1,2
)


select 
 cb.*, fla.digitalLoanAccountId,--onboarding_datetime, 
 fla.loantype,
case when lc.custid is not null then 1 else 0 end as flg_loan_calculator,
case when fla.poi1RejectType is not null or fla.loantype is null then 1 else 0 end as flg_reject_rule_check,
case when (fla.isTermsAndConditionstimestamp is not null or loan.new_loan_type ='Flex-up')  then 1 else 0 end as flg_applied_loan,
case when ul.custid is not null then 1 else 0 end as flg_approved_scorecard_loan,
case when fla.loanaccountnumber is not null then 1 else 0 end as flg_disbursed_loan,
fla.loanaccountnumber as loan_id
from cust_base cb
left join first_loan_application fla
on cast(cb.cust_id as integer)=fla.custId
left join loan_calculator lc
--on cast(cb.cust_id as integer)=lc.custid and 
on lc.digitalLoanAccountId=fla.digitalLoanAccountId
left join disbursed_undisbursed_loan ul
--on cast(cb.cust_id as integer)=ul.custid)
on ul.digitalLoanAccountId=fla.digitalLoanAccountId
left join 
risk_credit_mis.loan_master_table loan on loan.digitalLoanAccountId=fla.digitalLoanAccountId),                                                                                                  


default_outstanding_principal as (with all_default_account as (select digitalLoanAccountId,loanAccountNumber,firstDueDate,secondDueDate,thirdDueDate,defFPD30,defSPD30,defTPD30 from 
`risk_credit_mis.loan_master_table`
where defFPD10=1 or defFPD30=1 or defSPD30=1 or defTPD30=1),

defpd10_pricipal_outstanding as (
select loanAccountNumber,firstDueDate,PRINCIPALARREARS defpd10_outstanding_principal from `risk_credit_mis.loan_master_table`  loan
join core_raw.loan_accounts core on core.ACCOUNTNUMBER=loan.loanAccountNumber and (Date_add(firstDueDate ,INTERVAL 10 DAY)=core._Partitiondate or Date_add(firstDueDate ,INTERVAL 11 DAY)=core._Partitiondate)
where defFPD10=1 qualify row_number() over (partition by loanAccountNumber order by defpd10_outstanding_principal desc )=1),

defpd30_pricipal_outstanding as (
select loanAccountNumber,firstDueDate,PRINCIPALARREARS defpd30_outstanding_principal from `risk_credit_mis.loan_master_table`  loan
join core_raw.loan_accounts core on core.ACCOUNTNUMBER=loan.loanAccountNumber and (Date_add(firstDueDate ,INTERVAL 30 DAY)=core._Partitiondate or Date_add(firstDueDate ,INTERVAL 31 DAY)=core._Partitiondate)
where defFPD30=1 qualify row_number() over (partition by loanAccountNumber order by defpd30_outstanding_principal desc )=1),

defspd30_pricipal_outstanding as (
select loanAccountNumber,secondDueDate,PRINCIPALARREARS defspd30_outstanding_principal from `risk_credit_mis.loan_master_table`  loan
join core_raw.loan_accounts core on core.ACCOUNTNUMBER=loan.loanAccountNumber and (Date_add(secondDueDate ,INTERVAL 30 DAY)=core._Partitiondate or Date_add(secondDueDate ,INTERVAL 31 DAY)=core._Partitiondate)
where defSPD30=1 qualify row_number() over (partition by loanAccountNumber order by defspd30_outstanding_principal desc )=1),

deftpd30_pricipal_outstanding as (
select loanAccountNumber,thirdDueDate,PRINCIPALARREARS deftpd30_outstanding_principal from `risk_credit_mis.loan_master_table`  loan
join core_raw.loan_accounts core on core.ACCOUNTNUMBER=loan.loanAccountNumber and (Date_add(thirdDueDate ,INTERVAL 30 DAY)=core._Partitiondate or Date_add(thirdDueDate ,INTERVAL 31 DAY)=core._Partitiondate)
where defTPD30=1 qualify row_number() over (partition by loanAccountNumber order by deftpd30_outstanding_principal desc )=1)

select digitalLoanAccountId,allaccount.loanAccountNumber,defpd10_outstanding_principal,defpd30_outstanding_principal,defspd30_outstanding_principal,deftpd30_outstanding_principal,
allaccount.firstDueDate,allaccount.secondDueDate,allaccount.thirdDueDate,defFPD30 loan_defFPD30,defSPD30 loan_defSPD30,defTPD30 loan_defTPD30  from all_default_account allaccount
left join defpd10_pricipal_outstanding day10dpd on allaccount.loanAccountNumber=day10dpd.loanAccountNumber
left join defpd30_pricipal_outstanding firstdpd on allaccount.loanAccountNumber=firstdpd.loanAccountNumber
left join defspd30_pricipal_outstanding seconddpd on allaccount.loanAccountNumber=seconddpd.loanAccountNumber
left join deftpd30_pricipal_outstanding thirddpd on allaccount.loanAccountNumber=thirddpd.loanAccountNumber),

osversion_credolab as (select * from (
(select distinct deviceId,'android'||generalInfo.release as credolab_osversion ,GeneralInfo.model onb_model from credolab_raw.android_credolab_datasets_struct_columns
union all 
select distinct deviceId,'ios'||generalInfo.release as credolab_osversion ,GeneralInfo.model onb_model from  credolab_raw.ios_credolab_datasets_struct_columns))),

ln_model_credolab as (select * from (
(select distinct deviceId, GeneralInfo.model ln_model  from credolab_raw.android_credolab_datasets_struct_columns
union all 
select distinct deviceId,GeneralInfo.model ln_model from  credolab_raw.ios_credolab_datasets_struct_columns))),

onb_self_dec_income as 
(select custmtb.cust_id,monthly_income,company_name from 
`prj-prod-dataplatform.dl_customers_db_raw.tdbk_customer_mtb`  custmtb
join `prj-prod-dataplatform.dl_customers_db_raw.tdbk_cust_profile_mtb` profile on custmtb.user_id=profile.user_id
where monthly_income is not null),

cust_closed_date as (select OFCUSTOMERID,OFDATEOPENED, IF(OFDATECLOSED='1970-01-01' , NULL, OFDATECLOSED) OFDATECLOSED from `prj-prod-dataplatform.core_raw.customer_accounts`
join `dl_customers_db_raw.tdbk_customer_mtb` mtb on mtb.cust_id=OFCUSTOMERID
 where  CRINTERCODE = 'POC20000'
qualify row_number() over (partition by OFCUSTOMERID order by OFDATECLOSED desc) =1 and mtb.status='C'),

                                 
credolab_creation as (select --min(A.created_dt),min(B.createdOn)
digitalLoanAccountId,credolabRefNumber,A.userId,loanDigitalAppliedDateAndTime,A.created_dt,B.createdOn,A.clCreditScore,B.creditScore 
from `dl_loans_db_raw.tdbk_digital_loan_application` A
join dl_loans_db_raw.tdbk_credolab_track B on refno=credolabRefNumber qualify row_number() over(partition by credolabRefNumber,digitalLoanAccountId order by createdOn desc )=1),

credo_score_static_insght as (select * from (select 'static' as source, loan.digitalLoanAccountId,reference_number referenceNumber,datetime(credo_inquiry_date) requestedDate,null score_all_score,generic_score score_all_probability,
null score_sil_score,sil_score score_sil_probability, null score_quick_score,quick_score score_quick_probability,null score_flex_score,flexup_score score_flex_probability,null score_reloan_score, reloan_score score_reloan_probability from worktable_data_analysis.dema_score_final_may12 dima
left join risk_credit_mis.loan_master_table loan on dima.reference_number=loan.credolabRefNumber

union all

select 'credotrace' as source, loan.digitalLoanAccountId,referenceNumber referenceNumber,datetime(requestedDate) requestedDate,score_all_score,cast (score_all_probability as float64) score_all_probability,
score_sil_score,cast (score_sil_probability as float64) score_sil_probability,score_quick_score,cast(score_quick_probability as float64) score_quick_probability,score_flex_score,cast(score_flex_probability as float64) score_flex_probability,score_reloan_score,cast(score_reloan_probability as float64) score_reloan_probability from risk_credit_mis.credolab_trace_insight_all_loans  dima
left join risk_credit_mis.loan_master_table loan on dima.referenceNumber=loan.credolabRefNumber

union all
select 'insight' as source , digitalLoanAccountId,referenceNumber,datetime(requestedDate) requestedDate,score_all_score,cast (score_all_probability as float64) score_all_probability,
score_sil_score,cast (score_sil_probability as float64) score_sil_probability,score_quick_score,cast(score_quick_probability as float64) score_quick_probability,score_flex_score,cast(score_flex_probability as float64) score_flex_probability,score_reloan_score,cast(score_reloan_probability as float64) score_reloan_probability  from 

(
(select digitalLoanAccountId,referenceNumber,datetime(requestedDate) requestedDate,score_all_score,score_all_probability,
score_sil_score,score_sil_probability,score_quick_score,score_quick_probability,score_flex_score,score_flex_probability,score_reloan_score,score_reloan_probability from credolab_raw.credolab_insights_android
union all 
select digitalLoanAccountId,referenceNumber,requestedDate,score_all_score,score_all_probability,
score_sil_score,score_sil_probability,score_quick_score,score_quick_probability,score_flex_score,score_flex_probability,score_reloan_score,score_reloan_probability from credolab_raw.credolab_insights_ios))
qualify row_number() over(partition by digitalLoanAccountId order by requestedDate desc)  =1)
qualify row_number() over(partition by digitalLoanAccountId order by requestedDate)  =1)

select loanmaster.customerId,
cust_mtb.email as email,
cust_mtb.email_status as email_verified_flag,
cust_mtb.birthplace as place_of_birth,
cust_profile.first_nm first_name,
cust_profile.middle_nm middle_name,
cust_profile.last_nm last_name,
hm_city as onb_city,
hm_province as onb_province,
hm_postalcode as onb_postalcode,
hm_barangay as onb_barangay,
hm_country as onb_country,
latitude onb_latitude,
longitude onb_longitude,
DATE_DIFF(DATE(datetime(cust_mtb.created_dt,'Asia/Manila')), parse_date('%d %b %Y', cust_mtb.dob), YEAR) - IF(EXTRACT(MONTH FROM parse_date('%d %b %Y', cust_mtb.dob))*100 + EXTRACT(DAY FROM parse_date('%d %b %Y', cust_mtb.dob)) > EXTRACT(MONTH FROM DATE(datetime(cust_mtb.created_dt,'Asia/Manila')))*100 + EXTRACT(DAY FROM DATE(datetime(cust_mtb.created_dt,'Asia/Manila'))),1,0) as onb_age,
DATE_DIFF(DATE(if (loanmaster.new_loan_type ='Flex-up',loanmaster.startApplyDateTime,loanmaster.termsAndConditionsSubmitDateTime)), parse_date('%d %b %Y', cust_mtb.dob), YEAR) - IF(EXTRACT(MONTH FROM parse_date('%d %b %Y', cust_mtb.dob))*100 + EXTRACT(DAY FROM parse_date('%d %b %Y', cust_mtb.dob)) > EXTRACT(MONTH FROM DATE(if (loanmaster.new_loan_type ='Flex-up',loanmaster.startApplyDateTime,loanmaster.termsAndConditionsSubmitDateTime)))*100 + EXTRACT(DAY FROM DATE(if (loanmaster.new_loan_type ='Flex-up',loanmaster.startApplyDateTime,loanmaster.termsAndConditionsSubmitDateTime))),1,0) as loan_age,
cust_mtb.gender,
if (user_type_min.user_type='1_Repeat Applicant' or (new_loan_type in ('Flex-up') and decision_date is not null),1,0) onb_cnt_ongoing_loans,
loanmaster.digitalLoanAccountId,loanmaster.loanAccountNumber,datetime(cust_mtb.created_dt,'Asia/Manila') onboarding_datetime,
cust_mtb.mobile_no onb_mobile_no, mobileNo loan_mobile_no,case when length(digitalloan.alternateNumber)=10 then concat('63',digitalloan.alternateNumber) else digitalloan.alternateNumber end loan_alternate_mobile_no,loanmaster.purposeDescription loan_purpose,
source_fund.description AS source_funds,
emp_status.description AS employment_type,
nat.description AS nature_of_work,
industry.value industry_description,
loanmaster.companyName loan_companyName,
loanmaster.maritalStatus loan_marital_status,
loanmaster.dependentsCount loan_dependents_count,
educationtype.education_level loan_education_level,
if (loanmaster.new_loan_type ='Flex-up',loanmaster.startApplyDateTime,loanmaster.termsAndConditionsSubmitDateTime) loan_submission_date,
cust_mtb.kyc_status,
cust_mtb.kyc_status_change_dt,
loan_ref_type1,loan_ref_type2,
loanmaster.disbursementDateTime,
loan_level.addressline,
loan_level.province,
loan_level.city,
loan_level.barangay,
loan_level.postalcode,
loanaddress.full_address loan_address,
loanaddress.province loan_province,
loanaddress.city loan_city,
loanaddress.barangay loan_barangay,
loanaddress.postal_code loan_postal_code,
SPLIT(loan_level.geolocation, ' | ')[SAFE_OFFSET(0)] AS loan_latitude,
SPLIT(loan_level.geolocation, ' | ')[SAFE_OFFSET(1)] AS loan_longitude,
loan_level.docType loan_docType,
loan_level.docNumber loan_docNumber,
user_type_min.user_type,
user_type_loan_level.user_type ln_loan_level_user_type,
user_type_loan_level_v1.user_type ln_loan_level_user_type_v1,
loanmaster.new_loan_type,
case when loanmaster.new_loan_type ='Flex-up' and reloan_flag=1 then 'Reloan' else new_loan_type end loan_type,
    case when loanmaster.loantype='BNPL' and store_type =1 then 'Appliance'
    when loanmaster.loantype='BNPL' and store_type =2 then 'Mobile' 
    when loanmaster.loantype='BNPL' and store_type =3 then 'Mall' 
    when loanmaster.loantype='BNPL' and store_type not in (1,2,3) then store_tagging
    else 'not applicable' end as loan_product_type,
loanmaster.osversion_v2 osversion_v2,
if(lower(loanmaster.osversion_v2) like 'ios%','Apple', loan_level.brand) brand,
if(lower(loanmaster.osversion_v2) like 'ios%','iOS','Android') ln_os_type,
loanmaster.monthlyIncome loan_self_dec_income,
incomeperSalaryScale loan_salary_scaled_income,
case when vasflag.digitalLoanAccountId is not null then 1 else 0 end vas_used_flag,
if(poi3.crifApplicationId is not null,1,0) flg_cic_called,
flagmatched.matched_flag flg_cic_hit,
summary.ScoreRange,
--summary_matchedflag flg_cic_hit,
loanmaster.demographicScore demo_score,poi3.cicScorecard cic_score,loanmaster.credolabScore credo_score,trustingsocial.score trusting_social_score,poi3.bureauScore bureau_score,
--quick_score c_credo_score,sil_score s_credo_score,flexup_score fu_credo_score,reloan_score r_credo_score,generic_score gen_credo_score,
score_quick_probability c_credo_score,score_sil_probability s_credo_score,score_flex_probability fu_credo_score,score_reloan_probability r_credo_score,score_all_probability gen_credo_score,
if (user_type_min.user_type='1_Repeat Applicant' or (new_loan_type in ('Flex-up') and decision_date is not null),loanaccts.monthlytermamount,0) as onb_tot_ongoing_loans_emi,
empstatus.employment_type_new,
deliquency.flg_mature_fpd10,
deliquency.flg_mature_fpd30,
deliquency.flg_mature_fspd_30,
deliquency.flg_mature_fstpd_30,
deliquency.deffpd10,
deliquency.deffpd30,
deliquency.deffspd30,
deliquency.deffstpd30,
if(digitalloan.vas_flag='true',1,0) vas_opted_flag,
cust_profile.fatca,
flg_reject_rule_check,
flg_applied_loan,
--flg_approved_scorecard_loan,
flg_disbursed_loan,
CASE WHEN cddRejectReason IS NOT NULL and cddStatus <> 'ACTIVATED' THEN 1 ELSE 0 END flg_cdd_reject_flag,
CASE WHEN rejectedStage = 'Stage0' THEN 1 ELSE 0 END flg_hard_reject_flag, 
CASE WHEN rejectedStage = 'Stage1' THEN 1 ELSE 0 END flg_taran_reject_flag, 
case when digitalloan.poi3RejectType in ('SOFT','HARD') and digitalloan.poi3StatusCode like 'RJ%' then 1 else 0 end ln_taran_rule_reject_flag,
case when poi3.uw_flag in ('Streamline UW','Full UW') then 1 else 0 end ln_marked_underwriter_check_flag,

-- CASE WHEN cddRejectReason IS NOT NULL and cddStatus = 'ACTIVATED' THEN 1 ELSE 0 END flg_cdd_approved_flag,
-- CASE WHEN rejectedStage <> 'Stage0' THEN 1 ELSE 0 END flg_not_hard_uw_approved, 
-- CASE WHEN rejectedStage <> 'Stage1' THEN 1 ELSE 0 END flg_taran_approved_flag, 

CASE WHEN loanmaster.applicationstatus='APPROVED' THEN 1 ELSE 0 END flg_loan_approved_not_disbursed,
digitalloan.loanRequestAmount loan_chosen_principal,digitalloan.loanRequestTenure loan_chosen_tenor,
coalesce(disbursedLoanAmount,if(loanmaster.applicationstatus='APPROVED',loanmaster.approvedLoanAmount,NULL)) loan_approved_principal,
coalesce(disbursedLoanTenur,if(loanmaster.applicationstatus='APPROVED',loanmaster.approvedLoanTenure,NULL)) loan_approved_tenor,
loan_defFPD30,loan_defSPD30,loan_defTPD30,defpd10_outstanding_principal,defpd30_outstanding_principal,defspd30_outstanding_principal,deftpd30_outstanding_principal,loanmaster.credolabRefNumber,
loanmaster.docType,

coalesce(osversion_credolab.credolab_osversion,osversion) onb_osversion,  
coalesce(ln_model_credolab.ln_model,osversion_credolab.onb_model) ln_model,        
cust_mtb.status,
onb_self_dec_income.monthly_income onb_self_dec_income,
onb_self_dec_income.company_name onb_company_name,
cust_closed_date.OFDATECLOSED cust_status_close_date,
credolab_creation.createdOn credo_inquiry_date,
purpose.source_of_fund_name purpose_source_of_fund_name,purpose.nature_of_work_name purpose_nature_of_work_name,purpose.employment_status_name purpose_employment_status_name,purpose.industryName purpose_industryName,purpose.industryid pupose_industryid,purpose.created_dt purpose_created_dt,
loan_level.industry,loan_level.natureofwork,
loan_level.employmentstatus,
loan_level.source_of_funds,
ln_mail.email ln_email
                                    
--CASE WHEN loanPaidStatus in ('Normal','In Arrears') THEN 1 ELSE 0 END count_ongoing_loans 

from risk_credit_mis.loan_master_table loanmaster
inner join `dl_loans_db_raw.tdbk_digital_loan_application` digitalloan on loanmaster.digitalLoanAccountId=digitalloan.digitalLoanAccountId
left join `dl_customers_db_raw.tdbk_customer_mtb` cust_mtb on cast(loanmaster.customerId as string)=cust_mtb.cust_id
left join dl_loans_db_raw.tdbk_digital_loan_application digital on digital.digitalLoanAccountId=loanmaster.digitalLoanAccountId
left join dl_loans_db_raw.tdbk_loan_purpose purpose on purpose.loanPurposeId=digital.loanPurposeId
left join cust_info on cust_info.digitalLoanAccountId=loanmaster.digitalLoanAccountId
LEFT JOIN dl_dynamo_db_raw.tdbk_nature_of_work_mtb nat
ON cast(nat.id as string) = SPLIT(cust_mtb.is_employee,'|')[SAFE_OFFSET(3)]
LEFT JOIN dl_dynamo_db_raw.tdbk_employment_status_mtb emp_status
ON cast(emp_status.id as string) = SPLIT(cust_mtb.is_employee,'|')[SAFE_OFFSET(0)]
LEFT JOIN dl_dynamo_db_raw.tdbk_source_of_funds_mtb source_fund
ON cast(source_fund.id as string) = SPLIT(cust_mtb.is_employee,'|')[SAFE_OFFSET(4)]
LEFT JOIN dl_dynamo_db_raw.tdbk_industry_list_mtb industry
ON cast(industry.id as string) = SPLIT(cust_mtb.is_employee,'|')[SAFE_OFFSET(6)]
left join educationtype on educationtype.digitalLoanAccountId=digitalloan.digitalLoanAccountId
left join ref_type on ref_type.digitalLoanAccountId=loanmaster.digitalLoanAccountId
left join `risk_credit_mis.loan_level_master_table` loan_level on loan_level.digitalLoanAccountId=loanmaster.digitalLoanAccountId
left join (SELECT * FROM `prj-prod-dataplatform.dl_loans_db_raw.tdbk_vas_holiday_htb`  where  req_type=1
qualify row_number() over (partition by digitalLoanAccountId order by repayment_number desc)=1) vasflag on loanmaster.digitalLoanAccountId=vasflag.digitalLoanAccountId
LEFT JOIN (SELECT customer_id,    latitude,    longitude,  FROM    `risk_mart.customer_gps_location`
  WHERE event_description = 'Apigee Logs' AND latitude <> 'undefined' AND longitude <> 'undefined' 
  QUALIFY ROW_NUMBER() OVER (PARTITION BY customer_id order by gps_collection_date desc ) =1 ) loc on cast(loc.customer_id as int64)=loanmaster.customerId
  LEFT JOIN (select digitalLoanAccountId,max(installmentAmount) monthlytermamount from `risk_credit_mis.loan_installments_table` instal
--join core_raw_staging.loan_accounts loanacct on instal.loanAccountNumber=loanacct.ACCOUNTNUMBER  where LOANSTATUS in ('Normal','In Arrears')  
group by 1 ) loanaccts on loanaccts.digitalLoanAccountId = loanmaster.digitalLoanAccountId 
left join user_type_min on cast(user_type_min.customerId as string)=cust_mtb.cust_id 
left join user_type_loan_level on user_type_loan_level.digitalLoanAccountId=loanmaster.digitalLoanAccountId 
left join user_type_loan_level_v1 on user_type_loan_level_v1.digitalLoanAccountId=loanmaster.digitalLoanAccountId
left join worktable_data_analysis.lov_sf_industry_now_emp_status_apl15 empstatus on empstatus.digitalLoanAccountId=loanmaster.digitalLoanAccountId
  left join(SELECT DISTINCT mer_refferal_code, mer_name mer_name,store_type,store_tagging FROM `dl_loans_db_raw.tdbk_merchant_refferal_mtb`
  left join worktable_datachampions.TARGET_SPLIT P on P.STORE_NAME = mer_name
 qualify row_number() over(partition by mer_refferal_code order by  created_dt desc)=1) sil_category on loanmaster.purpleKey=sil_category.mer_refferal_code
  LEFT JOIN dl_customers_db_raw.tdbk_cust_profile_mtb cust_profile on cust_profile.user_id = cust_mtb.user_id
  left join dl_customers_db_raw.tdbk_customer_add_mtb  per_addr on per_addr.user_id = cust_mtb.user_id
   left join deliquency on loanmaster.loanAccountNumber=deliquency.loanAccountNumber
   left join default_outstanding_principal on default_outstanding_principal.digitalLoanAccountId=loanmaster.digitalLoanAccountId
   left join `dl_loans_db_raw.tdbk_loan_poi3_response` poi3 on loanmaster.crifApplicationId=poi3.crifApplicationId
left join (SELECT digitalLoanAccountId,score FROM `dl_loans_db_derived.tdbk_loan_poi3_response_trustingsocial_data` 
qualify row_number() over (partition by digitalLoanAccountId order by timestamp_received desc)=1) trustingsocial on 
loanmaster.digitalLoanAccountId=trustingsocial.digitalLoanAccountId
 left join (select digitalLoanAccountId, MAX(coalesce(cast(FlagMatched as int64),0)) as matched_flag from dl_loans_db_derived.tdbk_loan_poi3_response_bureau_string_xml_parsed_matchedsubject
group by 1) flagmatched
on loanmaster.digitalLoanAccountId = flagmatched.digitalLoanAccountId
left join 
(SELECT digitalLoanAccountId,if(substr(ScoreRange,1,3)='NH_',0,1) summary_matchedflag,ScoreRange FROM `dl_loans_db_derived.tdbk_loan_poi3_response_bureau_string_xml_parsed_summary` 
qualify row_number() over (partition by digitalLoanAccountId order by run_date desc)=1) summary 
on summary.digitalLoanAccountId = loanmaster.digitalLoanAccountId

--only applied loans 2 condition's has to check 
left join credo_score_static_insght credo_score_dima on credo_score_dima.digitalLoanAccountId=loanmaster.digitalLoanAccountId
--worktable_data_analysis.dema_score_final_may12 credo_score_dima  on 
--credo_score_dima.reference_number=loanmaster.credolabRefNumber 
--and loanmaster.credolabRefNumber is not null
left join (select userId,refno,createdOn from  `dl_loans_db_raw.tdbk_credolab_track`  qualify  row_number() over (partition by userId order by createdOn)=1) onb_osversion 
on onb_osversion.userId=digitalloan.userid
left join osversion_credolab on onb_osversion.refno=osversion_credolab.deviceid
left join ln_model_credolab on loanmaster.credolabRefNumber=ln_model_credolab.deviceid
left join onb_self_dec_income on onb_self_dec_income.cust_id=cast(loanmaster.customerId as string)
left join cust_closed_date on cust_closed_date.OFCUSTOMERID=cust_mtb.cust_id 
left join credolab_creation on credolab_creation.digitalLoanAccountId=loanmaster.digitalLoanAccountId
left join datalake_worktables.customer_mobile_mail_dtls ln_mail on ln_mail.cust_id=cast(loanmaster.customerId as string)  
and (coalesce(loanmaster.termsAndConditionsSubmitDateTime,if (loanmaster.new_loan_type ='Flex-up',loanmaster.startApplyDateTime,loanmaster.termsAndConditionsSubmitDateTime)) between DATETIME(ln_mail.valid_from_dt)   and  DATETIME(ln_mail.valid_to_dt))
left join (select * from `prj-prod-dataplatform.dl_loans_db_raw.tdbk_customer_loan_address` 

where status=1 qualify row_number() over (partition by digital_loan_account_id order by created_dt desc)=1) loanaddress on loanaddress.digital_loan_account_id=loanmaster.digitalLoanAccountId
                                 
where coalesce(loanmaster.termsAndConditionsSubmitDateTime,if (loanmaster.new_loan_type ='Flex-up',loanmaster.startApplyDateTime,loanmaster.termsAndConditionsSubmitDateTime)) between '2023-01-01' and '{OBDATE}' and  loanmaster.credolabRefNumber is not null) main
--where digitalLoanAccountId in (select digitalLoanAccountId from risk_mart.applied_loans_20230101_20250615)

qualify row_number() over(partition by main.digitalLoanAccountId)=1
;
"""
query_job = client.query(query)
# Wait for the job to complete.
query_job.result()
print(f"Created table: {query_job.destination}")

Created table: prj-prod-dataplatform.dap_ds_poweruser_playground.applied_loans


In [ ]:
sq = """select * from prj-prod-dataplatform.risk_mart.applied_loans;"""

from google.cloud import bigquery_storage
bq_storage_client = bigquery_storage.BigQueryReadClient()
dfd = client.query(sq).to_dataframe(bqstorage_client=bq_storage_client, progress_bar_type='tqdm')
print(f"The shape of applied_loans for {OBDATE} is:\t {dfd.shape}")

Job ID 62f413ec-ca94-406c-b2e8-0886a2a03d81 successfully executed: 100%|██████████|
Downloading: 100%|██████████|


# save_df_to_gcs

In [5]:
def save_df_to_gcs(df, bucket_name, destination_blob_name, file_format='csv'):
    """Saves a pandas DataFrame to Google Cloud Storage.

    Args:
        df: The pandas DataFrame to save.
        bucket_name: The name of the GCS bucket.
        destination_blob_name: The name of the blob to be created.
        file_format: The file format to save the DataFrame in ('csv' or 'parquet').
    """

    # Create a temporary file
    if file_format == 'csv':
        temp_file = 'temp.csv'
        df.to_csv(temp_file, index=False)
    elif file_format == 'parquet':
        temp_file = 'temp.parquet'
        df.to_parquet(temp_file, index=False)
    else:
        raise ValueError("Invalid file format. Please choose 'csv' or 'parquet'.")

    # Upload the file to GCS
    storage_client = storage.Client(project="prj-prod-dataplatform")

    storage_client = storage.Client()
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(destination_blob_name)

    blob.upload_from_filename(temp_file)

    # Remove the temporary file
    import os
    os.remove(temp_file)
    


# save_pickle_to_gcs

In [6]:
import pickle
import io
from google.cloud import storage
def save_pickle_to_gcs(data, bucket_name, destination_blob_name):
    """
    Save any Python object as a pickle file to Google Cloud Storage
    
    Args:
        data: The Python object to pickle (DataFrame, dict, list, etc.)
        bucket_name: Name of the GCS bucket
        destination_blob_name: Path/filename in the bucket
    """
    # Initialize the GCS client
    client = storage.Client()
    bucket = client.bucket(bucket_name)
    blob = bucket.blob(destination_blob_name)
    
    # Serialize the data to pickle format in memory
    pickle_buffer = io.BytesIO()
    pickle.dump(data, pickle_buffer)
    pickle_buffer.seek(0)
    
    # Upload the pickle data to GCS
    blob.upload_from_file(pickle_buffer, content_type='application/octet-stream')
    print(f"Pickle file uploaded to gs://{bucket_name}/{destination_blob_name}")

# Read the Data

In [7]:
bucket_name = BUCKET_NAME
csv_filename = f"applied_loans_20230101_{OBDATE}"
# Construct the new filename
data_filename = f"{csv_filename}.csv"
print(data_filename)
destination_blob_name = f"{CLOUDPATH}/{data_filename}"
save_df_to_gcs(dfd, bucket_name, destination_blob_name)

# Your modified code
bucket_name = BUCKET_NAME
pickle_filename = f"applied_loans_20230101_{OBDATE}"

# Construct the new filename with .pkl extension
data_filename = f"{pickle_filename}.pkl"
print(data_filename)

destination_blob_name = f"{CLOUDPATH}/{data_filename}"

# Save the DataFrame (or any object) as pickle to GCS
save_pickle_to_gcs(dfd, bucket_name, destination_blob_name)

prj-prod-dataplatform.risk_mart.applied_loans_20230101_2025-12-23.csv


ValueError: Stream is already exhausted. There is no content remaining.